In [ ]:
import pandas as pd
import numpy as np
import re
import ast

In [ ]:
data = pd.read_csv('updated_data.csv', encoding='utf-8-sig')

In [ ]:
# Kiểm tra số cột và dòng của dữ liệu
print(data.shape)

In [ ]:
# Kiểm tra tên các cột của dữ liệu
print(data.columns)

# Chuyển đổi kiểu dữ liệu

In [ ]:
# Kiểu dữ liệu của thuộc tính
data.info()

In [ ]:
# Chuyển đổi cột 'price_old' từ object sang numeric
data['price_old'] = data['price_old'].astype(str)
data['price_old'] = data['price_old'].str.replace('.', '', regex=False)
data['price_old'] = data['price_old'].str.replace('₫', '', regex=False)
data['price_old'] = data['price_old'].str.replace('đ', '', regex=False)
data['price_old'] = pd.to_numeric(data['price_old'], errors='coerce')

In [ ]:
# Chuyển đổi cột 'price_new' từ object sang numeric
data['price_new'] = data['price_new'].astype(str)
data['price_new'] = data['price_new'].str.replace('.', '', regex=False)
data['price_new'] = data['price_new'].str.replace('₫', '', regex=False)
data['price_new'] = data['price_new'].str.replace('đ', '', regex=False)
data['price_new'] = pd.to_numeric(data['price_new'], errors='coerce')


# Xử lí cột brand

In [ ]:
print(data['brand'].unique())

Cần thống nhất nội dung

In [ ]:
# Xử lí cột 'brand'
data['brand'] = data['name'].str.split(' ').str[0]
data.loc[data['brand'] == 'iPhone', 'brand'] = 'iPhone (Apple)'

# Xử lí cột color và image

In [ ]:
print(data['color'].unique())

In [ ]:
print(data['image'].unique())

Với dữ liệu lấy từ cellphones thì color và image là một list gốm nhiều màu và ảnh khác nhau -> Xử lí cột 'color' và 'image' bằng cách: với mỗi màu trong 1 mẫu điện thoại tạo một dòng dữ liệu mới kèm với hình ảnh tương ứng.  
Với dữ liệu lấy từ thế giới di động, loại bỏ chuỗi "Màu: "

In [ ]:
# Loại bỏ chuỗi "Màu: " trong cột 'color'
data['color'] = data['color'].str.replace('Màu: ', '', regex=False)

In [ ]:
# Xử lý cột 'color' và 'image': với mỗi màu trong 1 mẫu điện thoại tạo một dòng dữ liệu mới kèm với hình ảnh tương ứng

# Khởi tạo các cột mới để lưu các list đã parse
data['parsed_color_list'] = [[] for _ in range(len(data))]
data['parsed_image_list'] = [[] for _ in range(len(data))]


for index, row in data.iterrows():
    # Xử lý cột 'color'
    color_str = str(row['color']).strip()
    
    if color_str.lower() in ['', 'nan', 'none']:
        parsed_colors = []
    elif color_str.startswith('[') and color_str.endswith(']'):
        parsed_list = ast.literal_eval(color_str)
        if isinstance(parsed_list, list):
            cleaned_colors = [str(c).strip() for c in parsed_list if str(c).strip() != '']
            parsed_colors = cleaned_colors
        else:
            parsed_colors = []
    else:
        parsed_colors = [c.strip() for c in color_str.split(',') if c.strip() != '']
    data.at[index, 'parsed_color_list'] = parsed_colors

    # Xử lý cột 'image'
    img_url_str = str(row['image']).strip()

    if img_url_str.lower() in ['', 'nan', 'none']:
        parsed_images = []
    elif img_url_str.startswith('[') and img_url_str.endswith(']'):
        parsed_list = ast.literal_eval(img_url_str)
        if isinstance(parsed_list, list):
            cleaned_urls = [str(u).strip() for u in parsed_list if str(u).strip() != '']
            parsed_images = cleaned_urls
        else:
            parsed_images = []
    else:
        urls = [u.strip() for u in img_url_str.split(',') if u.strip() != '']
        parsed_images = urls
    data.at[index, 'parsed_image_list'] = parsed_images

# Tạo một cột tạm thời chứa các list of tuples
data['color_image_list_of_tuples'] = [[] for _ in range(len(data))]

for index, row in data.iterrows():
    colors = row['parsed_color_list']
    images = row['parsed_image_list']
    
    len_colors = len(colors)
    len_images = len(images)

    if len_colors == 0 and len_images == 0:
        data.at[index, 'color_image_list_of_tuples'] = [(np.nan, np.nan)]
    # Nếu chỉ có màu, điền NaN cho ảnh
    elif len_colors > 0 and len_images == 0:
        data.at[index, 'color_image_list_of_tuples'] = [(color, np.nan) for color in colors]
    # Nếu chỉ có ảnh, điền NaN cho màu
    elif len_images > 0 and len_colors == 0:
        data.at[index, 'color_image_list_of_tuples'] = [(np.nan, image) for image in images]
    # Độ dài không khớp:
    else:
        max_len = max(len_colors, len_images)
        temp_tuples = []
        for i in range(max_len):
            color_val = colors[i] if i < len_colors else np.nan
            image_val = images[i] if i < len_images else np.nan
            temp_tuples.append((color_val, image_val))
        data.at[index, 'color_image_list_of_tuples'] = temp_tuples

# Explode cột chứa list các tuple
data_exploded = data.explode('color_image_list_of_tuples')

# Loại bỏ các cột không cần thiết
data_exploded = data_exploded.drop(columns=[
    'image',
    'color',
    'parsed_color_list',
    'parsed_image_list',
], errors='ignore')

# Tạo các cột 'color' và 'image' mới từ cột tuple đã explode
data_exploded['color'] = data_exploded['color_image_list_of_tuples'].apply(lambda x: x[0])
data_exploded['image'] = data_exploded['color_image_list_of_tuples'].apply(lambda x: x[1])
data_exploded = data_exploded.drop(columns='color_image_list_of_tuples')
data = data_exploded.copy()

# Xử lí cột condition

In [ ]:
print(data['condition'].unique())

Cần thống nhất nội dung trong thuộc tính

In [ ]:
condition_normalization_map = {
    'cũ trầy xước': 'Cũ trầy xước',
    'cũ xước cấn': 'Cũ trầy xước cấn',
    'cũ đẹp': 'Cũ đẹp',
    'hàng trưng bày': 'Hàng trưng bày',
    'đã kích hoạt': 'Đã kích hoạt',
    'chính hãng (vn/a) đã kích hoạt': 'Đã kích hoạt (Chính Hãng VN/A)',
    'đã kích hoạt bảo hành vn/a': 'Đã kích hoạt (Bảo Hành VN/A)',
    'đổi bảo hành': 'Đổi bảo hành',
    'đổi bảo hành (vn/a)': 'Đổi Bảo Hành (VN/A)',
    'đổi bảo hành vn/a': 'Đổi Bảo Hành (VN/A)',
    'xước cấn': 'Cũ trầy xước cấn',
    'cũ': 'Cũ',
    'mới fullbox': 'Mới Fullbox',
    'like new 99%': 'Like New 99%',
    'chính hãng đã kích hoạt': 'Đã kích hoạt (Chính Hãng)',
    '256gb)': 'Còn nguyên dung lượng lưu trữ',
    '512gb)': 'Còn nguyên dung lượng lưu trữ',
    '1tb)': 'Còn nguyên dung lượng lưu trữ',
    '128gb)': 'Còn nguyên dung lượng lưu trữ',
    'có trầy xước': 'Cũ trầy xước'
}
condition = data['condition']
data['condition'] = data['condition'].astype(str).str.lower()
data['condition'] = data['condition'].map(condition_normalization_map) 
data['condition'] = data['condition'].fillna(condition)
data['condition'] = data['condition'].str.strip()

# Xử lí cột time

In [ ]:
print(data['time'].unique())

In [ ]:
def convert_date(date_str):
    try:
        date_obj = pd.to_datetime(date_str, errors='coerce', dayfirst=True)
        if pd.isna(date_obj):
            return date_str  
        return date_obj.strftime("%d/%m/%Y") 
    except Exception as e:
        return str(e)
    
data['time'] = data['time'].apply(convert_date)
data['time'] = pd.to_datetime(data['time'], format="%d/%m/%Y", errors='coerce')


# Xử lí cột CPU

In [ ]:
print(data['CPU'].unique())

Cần loại bỏ các thông tin không cần thiết, sửa lỗi chính tả.

In [ ]:
cpu = data['CPU']
data['CPU'] = data['CPU'].fillna('')
data['CPU'] = data['CPU'].str.split(',').str[0].str.strip()
data['CPU'] = data['CPU'].str.replace(r'\s*\(.*?\)\s*', ' ', regex=True)
data['CPU'] = data['CPU'].str.replace(r'[^\w\s\+\.]', ' ', regex=True)
data['CPU'] = data['CPU'].str.replace(r'\s+', ' ', regex=True).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bQualcomm\s+SM\d+\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\b(thế hệ|gen|GEN)\b', 'Gen', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bcore\b', 'nhân', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bQualcomm\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bthứ\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bMobile\s+Platform\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\b(dành cho|For)\b', 'for', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'^\s*(?:Chip|Mật độ)\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\s*(?:\d+\s*nm\+?|tăng)\s*.*$', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bnapdragon\b', 'Snapdragon', regex=True, flags=re.IGNORECASE).str.strip()

cleaned_cpus = []
for cpu_entry in data['CPU']:
    if pd.isna(cpu_entry) or cpu_entry == '':
        cleaned_cpus.append(np.nan)
        continue

    final_cpu_name = re.sub(r'(Gen)(\d+)', r'\1 \2', str(cpu_entry), flags=re.IGNORECASE)
    cleaned_cpus.append(final_cpu_name.strip())

data['CPU'] = cleaned_cpus
data['CPU'] = data['CPU'].fillna(cpu)

# Xử lí cột warranty

In [ ]:
print(data['warranty'].unique())

Cần loại bỏ các chuỗi không cần thiết trong cột warranty

In [ ]:
data['warranty'] = data['warranty'].str.replace(' tại TGDĐ', '', regex=False)
data['warranty'] = data['warranty'].str.replace('Chính hãng đến ', '', regex=False)

# Xử lí cột battery

In [ ]:
print(data['battery'].unique())

Dữ liệu trong cột battery không thống nhất theo đơn vị mAh và loại bỏ thông tin không cần thiết

## Với dữ liệu từ thế giới di động

In [ ]:
# Thống nhất dữ liệu trong cột 'battery' sang mAh
condition = data['battery'] == '27 giờ'
data.loc[condition, 'battery'] = '4674 mAh'
condition = data['battery'] == '26 giờ'
data.loc[condition, 'battery'] = '4005 mAh'
condition = data['battery'] == '22 giờ'
data.loc[condition, 'battery'] = '3561 mAh'
condition = data['battery'] == '33 giờ'
data.loc[condition, 'battery'] = '4685 mAh'

## Với dữ liệu từ cellphones

In [ ]:
# Thống nhất dữ liệu trong cột 'battery' sang mAh
data['battery'] = data['battery'].fillna('').astype(str).str.lower()
data['battery'] = data['battery'].str.replace(r'\s+', '', regex=True)
data['battery'] = data['battery'].str.replace(r'[,.]', '', regex=True)
extracted_mah = data['battery'].str.extract(r'(\d+)mah', flags=re.IGNORECASE, expand=False)
data['battery'] = pd.to_numeric(extracted_mah, errors='coerce')

# Lọc các sản phẩm có 'battery' là NaN sau chuyển đổi
products_with_nan_battery = data[data['battery'].isna()]
unique_names_with_nan_battery = products_with_nan_battery['name'].unique()
# In ra name của các sản phẩm đó
if unique_names_with_nan_battery.size > 0:
    print("\nSản phẩm có giá trị pin (mAh) là NaN:")
    for name in unique_names_with_nan_battery:
        print(f"- {name}")
else:
    print("\nKhông có sản phẩm nào có giá trị pin (mAh) là NaN.")

# Điền các trị NaN
lookup_data = {
    'product_name_lookup': ['iPhone 14 Pro Max', 'iPhone 14 Pro', 'iPhone 15 Plus', 'iPhone 15 Pro',
                            'iPhone 15 Pro Max', 'iPhone 16 Plus', 'iPhone XS Max', 'iPhone 12 Pro Max',
                            'iPhone 16 Pro Max', 'iPhone 15', 'iPhone 14 Plus', 'iPhone 12 Pro', 
                            'iPhone 12', 'iPhone 16 Pro', 'iPhone 16', 'iPhone XS', 'iPhone 12 mini',
                            'iPhone 13', 'realme 9i', 'iPhone 14', 'OPPO Reno 13 5G',
                            'Xiaomi Redmi Note 14', 'Sony Xperia 10V', 'OPPO Reno13 Pro',
                            'Xiaomi Poco X7 Pro 5G', 'Xiaomi Redmi 10C 4G', 'Xiaomi Redmi Note 14 Pro',
                            'Xiaomi Mi 11T'],
    'correct_battery': [4323, 3200, 4383, 3274, 4422, 4674, 3174, 3687, 4685, 3349, 4325, 2815, 2815,
                       4674, 3561, 2658, 2227, 3240, 5000, 3279, 5600, 5500, 5000, 5800, 6000, 5000,
                       5500, 5000]
}
lookup = pd.DataFrame(lookup_data)

battery_map = lookup.set_index('product_name_lookup')['correct_battery']
data['battery'] = data['battery'].fillna(data['name'].map(battery_map))

# Xử lí cột RAM và capacity

In [ ]:
print(data['RAM'].unique())

Dữ liệu trong cột RAM không thống nhất theo đơn vị GB.

In [ ]:
print(data['capacity'].unique())

Dữ liệu theo cột capacity không thống nhất theo đơn vị GB.

In [ ]:
# 1. Chuẩn hóa cột RAM: lấy phần trước "mở rộng" nếu có
data['RAM'] = data['RAM'].apply(lambda x: (
    f"{re.search(r'(\d+)\s*+', str(x).lower().split('mở rộng')[0]).group(1)} GB"
    if isinstance(x, str) and 'mở rộng' in x.lower() and re.search(r'(\d+)\s*+', x.lower().split('mở rộng')[0])
    else x
))

# 2. Hàm convert sang số GB
def convert_to_gb(value):
    if pd.isna(value) or str(value).lower().strip() == 'nan':
        return np.nan
    value = str(value).upper().strip()
    if 'GB' in value:
        try:
            return float(value.replace('GB', '').strip())
        except ValueError:
            return np.nan
    elif 'MB' in value:
        try:
            return float(value.replace('MB', '').strip()) / 1024
        except ValueError:
            return np.nan
    elif 'TB' in value:
        try:
            return float(value.replace('TB', '').strip()) * 1024
        except ValueError:
            return np.nan
    else:
        return np.nan

# 3. Áp dụng convert_to_gb cho cột RAM (và capacity nếu cần)
data['RAM'] = data['RAM'].apply(convert_to_gb)

if 'capacity' in data.columns and data['capacity'].dtype == object:
    data['capacity'] = data['capacity'].apply(convert_to_gb)

# Xử lí cột screen size

In [ ]:
print(data['screen_size'].unique())

Cần loại bỏ inches và ", chỉ lấy kích thước màn hình chính

In [ ]:
data['screen_size'] = data['screen_size'].apply(
    lambda x: x.split('&')[0].split('Chính')[1].strip() 
    if isinstance(x, str) and 'Chính' in x and len(x.split('Chính')) > 1
    else x if isinstance(x, str) else None
)

# Chuyển cột 'screen_size' sang kiểu dữ liệu số
data['screen_size'] = data['screen_size'].astype(str)
data['screen_size'] = data['screen_size'].str.replace(r'"', '', regex=True).str.strip()
data['screen_size'] = data['screen_size'].str.replace(r'\s*inches|\s*inch', '', regex=True)
data['screen_size'] = pd.to_numeric(data['screen_size'], errors='coerce')

# Xử lý cột screen resolution

In [ ]:
print(data['screen_resolution'].unique())

Cần loại bỏ thông tin không cần thiết, tách cột

In [ ]:
# Xử lí thông tin độ phân giải trong cột 'screen_resolution'
data['screen_resolution_temp'] = data['screen_resolution'].apply(
    lambda x: x.split('x P')[0].split('Chính: ')[1].strip()
    if isinstance(x, str) and 'Chính: ' in x
    else x if isinstance(x, str) else None
)

# Tách cột 'screen_resolution' thành 'resolution_width' và 'resolution_height'
data[['resolution_width', 'resolution_height']] = data['screen_resolution_temp'].str.extract(r'(?:.*\(|)(\d+) x (\d+)')

# Chuyển đổi cột 'resolution_width' sang kiểu số
data['resolution_width'] = data['resolution_width'].astype(str)
data['resolution_width'] = pd.to_numeric(data['resolution_width'], errors='coerce')

# Chuyển đổi cột 'resolution_height' sang kiểu số
data['resolution_height'] = data['resolution_height'].astype(str)
data['resolution_height'] = pd.to_numeric(data['resolution_height'], errors='coerce')

# Xóa các cột không cần thiết.
data = data.drop(columns=['screen_resolution_temp'])

# Xử lý cột operating system

In [ ]:
print(data['operating_system'].unique())

In [ ]:
data.loc[data['operating_system'] == '11', 'operating_system'] = 'iOS 11'
data.loc[data['operating_system'] == '12', 'operating_system'] = 'iOS 12'
# Loại bỏ phần sau dấu phẩy trong cột 'operating_system'
data['operating_system'] = data['operating_system'].str.split(',').str[0].str.strip()

In [ ]:
#Loại bỏ các chữ thừa, chỉ giữ lại tên và phiên bản
data['operating_system'] = data['operating_system'].str.extract(
    r'(iOS ?\d+(?:\.\d+)?|Android(?:™)? ?\d+(?:\.\d+)?|Android ?[UT]|Android ?\d+ ?Go|MIUI ?\d+(?:\.\d+)?|ColorOS ?\d+(?:\.\d+)?|Funtouch OS ?\d+(?:\.\d+)?|realme UI ?[\w\.]+|HarmonyOS ?\d+(?:\.\d+)?|Xiaomi HyperOS ?\d*|Mocor ?\(RTOS\)|XOS ?\d+(?:\.\d+)?|OxygenOS|One UI ?\d+(?:\.\d+)?|EMUI ?\d+(?:\.\d+)?|Series ?30\+|S30\+)',
    flags=re.IGNORECASE
)
data['operating_system'] = data['operating_system'].apply(
    lambda x: (
        re.match(r'^([A-Za-z]+(?: [A-Za-z]+)*)(?:\s?(\d+\.?\w*))?', 
                 re.sub(r'(Android)(\d)', r'\1 \2',
                 re.sub(r'(iOS)(\d)', r'\1 \2',
                 re.sub(r'\s+', ' ', re.sub(r'[™®]', '', x.strip())))))
        .group(0).strip()
        if isinstance(x, str) and re.match(r'^([A-Za-z]+(?: [A-Za-z]+)*)(?:\s?(\d+\.?\w*))?', x)
        else x
    )
)


In [ ]:
print(data['operating_system'].unique())

# Xử lý cột SIM

In [ ]:
print(data['SIM'].unique())

In [ ]:
# Xử lí thông tin cột 'SIM'
data['processed_SIM'] = data['SIM'].astype(str).str.lower()
data['processed_SIM'] = data['processed_SIM'].replace('nan', np.nan)

# Tạo cột 'has_nano_sim' (nếu có là 1 - không có là 0)
data['has_nano_sim'] = data['processed_SIM'].str.contains(r'nano[-]?sim', case=False, na=np.nan).astype(float)

# Tạo cột 'has_esim' (nếu có là 1 - không có là 0)
data['has_esim'] = data['processed_SIM'].str.contains(r'e[-]?sim', case=False, na=np.nan).astype(float)

# Cập nhật regex để bắt 'SIM' hoặc 'SIM 1', 'SIM 2',
mask_generic_sim = (
    data['processed_SIM'].notna() &
    data['processed_SIM'].str.contains(r'\bSIM(?: \d+| vật lý)?\b', case=False, na=False)
)

data = data.drop(columns = 'processed_SIM')

# Cập nhật has_nano_sim thành 1.0 cho các trường hợp này
data.loc[mask_generic_sim, 'has_nano_sim'] = 1.0

# Xử lý cột display technology

In [ ]:
print(data['display_technology'].unique())

Cần loại bỏ các thông tin không cần thiết 

In [ ]:
display_info = data['display_technology']
data['display_technology'] = data['display_technology'].fillna('').astype(str)
data['display_technology'] = data['display_technology'].str.extract(r'(.*?lcd)', flags=re.IGNORECASE, expand=False)
data['display_technology'] = data['display_technology'].fillna(display_info)
data['display_technology'] = data['display_technology'].str.split(',').str[0]
data['display_technology'] = data['display_technology'].str.replace('Cảm ứng điện dung ', '', regex=False)
data['display_technology'] = data['display_technology'].str.strip()

# Xử lý cột size

In [ ]:
print(data['size'].unique())

Chỉ lấy kích thước, tạo ra các cột mới lưu thông tin dài, rộng, cao

In [ ]:
# Xử lý dữ liệu cột 'size' để tạo 3 cột mới
data['size_str_processed'] = (data['size'].astype(str).str.strip()
                              .replace(['', 'nan', 'none'], 'INVALID', regex=True))

processed_size_lists = []
for size_item_str in data['size_str_processed']:
    current_list = [np.nan] * 3
    if size_item_str != 'INVALID':
        try:
            parsed_list = ast.literal_eval(size_item_str)
            if isinstance(parsed_list, list):
                numeric_elements = [pd.to_numeric(val, errors='coerce') for val in parsed_list]
                current_list = numeric_elements[:3] + [np.nan] * (3 - len(numeric_elements))
        except (ValueError, SyntaxError):
            pass
    processed_size_lists.append(current_list)

data[['height', 'width', 'depth']] = pd.DataFrame(processed_size_lists, index=data.index)
data.drop(columns=['size_str_processed'], inplace=True)

# Chuẩn hóa cột 'size'
data['general_size_info_cleaned'] = (data['size'].astype(str)
                                     .str.replace(',', '.', regex=False)
                                     .str.replace(r'\bxx\b', 'x', regex=True)
                                     .str.replace('≈', '', regex=False)
                                     .str.strip()
                                     .replace(['nan', 'none', ''], np.nan))

# Xử lý kích thước từ mô tả text
pattern_height = re.compile(r'(?:dọc|dài|chiều dài|cao)\s*[:\-]?\s*([\d\.]+)', re.IGNORECASE)
pattern_width = re.compile(r'(?:rộng|ngang|chiều rộng)\s*[:\-]?\s*([\d\.]+)', re.IGNORECASE)
pattern_depth = re.compile(r'(?:dày|độ dày|siêu mỏng)\s*[:\-]?\s*([\d\.]+)', re.IGNORECASE)
pattern_3_dims = re.compile(r'([\d\.]+)\s*(?:mm)?\s*[-xX×*]\s*([\d\.]+)\s*(?:mm)?\s*[-xX×*]\s*([\d\.]+)', re.IGNORECASE)

dims = data['general_size_info_cleaned'].str.extract(pattern_3_dims)
data['height'] = pd.to_numeric(dims[0], errors='coerce')
data['width'] = pd.to_numeric(dims[1], errors='coerce')
data['depth'] = pd.to_numeric(dims[2], errors='coerce')

data['height'] = data['height'].fillna(data['general_size_info_cleaned'].str.extract(pattern_height)[0].astype(float))
data['width'] = data['width'].fillna(data['general_size_info_cleaned'].str.extract(pattern_width)[0].astype(float))
data['depth'] = data['depth'].fillna(data['general_size_info_cleaned'].str.extract(pattern_depth)[0].astype(float))

data.drop(columns=['general_size_info_cleaned'], errors='ignore', inplace=True)

# Xử lý cột GPU

In [ ]:
print(data['GPU'].unique())

Loại bỏ các thông tin không cần thiết, sửa lỗi chính tả.

In [ ]:
gpu = data['GPU']
data['GPU'] = data['GPU'].fillna('')
data['GPU'] = data['GPU'].str.replace(r'[^\w\s\+\.\-]', ' ', regex=True)
data['GPU'] = data['GPU'].str.replace(r'\s+', ' ', regex=True).str.strip()
data['GPU'] = data['GPU'].str.replace(r'\bQualcomm\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['GPU'] = data['GPU'].str.replace(r'(\d+)\s*(?:lõi|core)s?\b', r'\1 nhân', regex=True, flags=re.IGNORECASE).str.strip()
data['GPU'] = data['GPU'].str.replace(r'\b(up to|Graphics|đồ họa|mới)\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['GPU'] = data['GPU'].str.replace(r'(\d+)-(?:nhân|lõi|core)s?\b', r'\1 nhân', regex=True).str.strip()
data['GPU'] = data['GPU'].str.replace(r'(\b\w+)\s*(-?G\d+)', r'\1\2', regex=True).str.strip()
data['GPU'] = data['GPU'].str.replace(r'\bAdreno(?:\s*\bGPU\b)?\s*(\d+)\b(?:\s*\bGPU\b)?', r'Adreno \1', regex=True, flags=re.IGNORECASE).str.strip()

cleaned_gpus = []
for gpu_entry in data['GPU']:
    if pd.isna(gpu_entry) or gpu_entry == '':
        cleaned_gpus.append(np.nan)
        continue

    processed_gpu_entry = str(gpu_entry)
    cleaned_gpus.append(processed_gpu_entry.strip())  

data['GPU'] = cleaned_gpus
data['GPU'] = data['GPU'].fillna(gpu)

# Xử lý cột bluetooth

In [ ]:
print(data['bluetooth'].unique())

Loại bỏ thông tin không cần thiết

In [ ]:
data['processed_bluetooth'] = data['bluetooth'].fillna('').astype(str)
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'(\d+),(\d+)', r'\1.\2', regex=True)
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'\bv\s*(\d+\.\d+)', r'v\1', regex=True, flags=re.IGNORECASE).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'(?<!Dual[\s\-])\bBluetooth\b\s*', '', regex=True).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'\baptX\b', 'apt-X', regex=True, flags=re.IGNORECASE).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'\bBT\s*(\d+\.\d+.*)?\b', r'\1', regex=True, flags=re.IGNORECASE).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'(?<!v)(\b\d+\.\d+(?:\s*[a-zA-Z0-9]+)?\b)', r'v\1', regex=True, flags=re.IGNORECASE).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'\b(Có)\b(?!$)', '', regex=True).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r',\s*', ', ', regex=True)
data['bluetooth'] = data['processed_bluetooth'].replace('', np.nan)
data = data.drop(columns=['processed_bluetooth'], errors='ignore')

# Xử lý cột refresh rate

In [ ]:
print(data['refresh_rate'].unique())

Lấy thông tin tần số quét chính, loại bỏ chữ Hz

In [ ]:
# Xử lí tần số quét chính & phụ
data['refresh_rate'] = data['refresh_rate'].astype(str).str.extract(
    r"Tần số quét Chính: (\d+) Hz", flags=re.IGNORECASE, expand=False
)

# Xử lý các hàng mà ở bước xử lí trên không tìm thấy
data['refresh_rate'] = data.apply(
    lambda row: re.search(r"Tần số quét (\d+) Hz", str(row['refresh_rate']), re.IGNORECASE).group(1)
                if pd.isna(row['refresh_rate']) and isinstance(row['refresh_rate'], str)
                   and re.search(r"Tần số quét (\d+) Hz", str(row['refresh_rate']), re.IGNORECASE)
                   and "Chính" not in str(row['refresh_rate'])
                else row['refresh_rate'],
    axis=1
)

# Chuyển cột 'refresh_rate' sang kiểu dữ liệu số
data['refresh_rate'] = data['refresh_rate'].astype(str)
data['refresh_rate'] = pd.to_numeric(data['refresh_rate'], errors='coerce')

# Lưu dữ liệu đã qua xử lí

In [ ]:
# Xuất DataFrame ra file CSV
data.to_csv('tidy_data.csv', index=False, encoding='utf-8-sig')

# Xử lý giá trị thiếu

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

đối với condition, warranty không thu thập được, mặc định là "Cũ" và "0 tháng" (không có bảo hành).

In [ ]:
data['condition'] = data['condition'].replace('', pd.NA).fillna('Cũ')
data['warranty'] = data['warranty'].replace('', pd.NA).fillna('0 tháng')

Cột price_new, CPU, RAM, capacity có chứa giá trị thiếu, nghi ngờ bị thiếu theo dòng máy.


In [ ]:
# price_new
data['price_new_missing'] = data['price_new'].isna().astype('int')
# Tính tỷ lệ missing price_new theo tên điện thoại
rate_prc_name = data.groupby('name')['price_new_missing'].mean()
print(rate_prc_name[rate_prc_name > 0])

data = data.drop(columns=['price_new_missing'])

Đối với một số dòng máy có tỷ lệ thiếu 100%, qua tra cứu thì các dòng máy trên đã ngừng kinh doanh sản phẩm mới -> không có giá bán mới. 
Xử lý: Đối với các máy còn lại có tỉ lệ thiếu >0, <100% thì điền thiếu bằng cách lấy giá bán mới trung bình của các mẫu cùng tên.
Do price_new là một thuộc tính quan trọng của bài toán -> drop những dòng bị missing ở thuộc tính price_new.

In [ ]:
for name_value, group in data.groupby('name'):
    total = len(group)
    missing = group['price_new'].isna().sum()
    ratio = missing / total

    if 0 < ratio < 1:
        # Lấy giá trị trung bình price_new trong nhóm
        price_value = round(group['price_new'].mean())
        # Gán lại cho các dòng bị thiếu trong nhóm đó
        data.loc[(data['name'] == name_value) & (data['price_new'].isna()), 'price_new'] = price_value

In [ ]:
data = data.dropna(subset=['price_new'])

In [ ]:
# CPU
data['CPU_missing'] = data['CPU'].isna().astype('int')
# Tính tỷ lệ missing CPU theo tên điện thoại
rate_CPU_name = data.groupby('name')['CPU_missing'].mean()
print(rate_CPU_name[rate_CPU_name > 0])

data = data.drop(columns=['CPU_missing'])

Chỉ thiếu phụ thuộc vào dòng máy (trên trang web bán hàng không công bố thông số) -> tìm kiếm và bổ sung. Đối với 3 dòng Itel it2600, Itel it9211, Itel it9310 thì nhà sản xuất không công bố rõ ràng về thông tin CPU -> không bổ sung. 

In [ ]:
data.loc[data['name'] == 'INOI 288S 4G', 'CPU'] = 'Unisoc T107'
data.loc[data['name'] == 'Nokia HMD 105 4G', 'CPU'] = 'Unisoc T127'
data.loc[data['name'] == 'Nubia Music NFC', 'CPU'] = 'Unisoc SC9863A'
data.loc[data['name'] == 'Samsung Galaxy A33', 'CPU'] = 'Exynos 1280'
data.loc[data['name'] == 'Sony Xperia 10V', 'CPU'] = 'Qualcomm Snapdragon 695 5G'

In [ ]:
# RAM
data['RAM_missing'] = data['RAM'].isna().astype('int')
# Tính tỷ lệ missing RAM theo tên điện thoại
rate_RAM_name = data.groupby('name')['RAM_missing'].mean()
print(rate_RAM_name[rate_RAM_name > 0])

data = data.drop(columns=['RAM_missing'])

Rõ ràng thiếu do phụ thuộc vào dòng máy. Xử lý: Tương tự CPU.

In [ ]:
data.loc[data['RAM'] == 'INOI 288S 4G', 'RAM'] = convert_to_gb('48 MB')
data.loc[data['RAM'] == 'Itel it9211', 'RAM'] = convert_to_gb('16 MB')
data.loc[data['RAM'] == 'Nokia HMD 105 4G', 'RAM'] = convert_to_gb('64 MB')
data.loc[data['RAM'] == 'iPhone 12 Pro', 'RAM'] = convert_to_gb('6 GB')
data.loc[data['RAM'] == 'iPhone 7 Plus', 'RAM'] = convert_to_gb('3 GB')

In [ ]:
#capacity
data['capacity_missing'] = data['capacity'].isna().astype('int')
# Tính tỷ lệ missing RAM theo tên điện thoại
rate_capacity_name = data.groupby('name')['capacity_missing'].mean()
print(rate_capacity_name[rate_capacity_name > 0])

data = data.drop(columns=['capacity_missing'])

Rõ ràng thiếu do phụ thuộc vào dòng máy. Xử lý: Tương tự CPU. 

In [ ]:
data.loc[data['capacity'] == 'INOI 288S 4G', 'capacity'] = convert_to_gb('128 MB')
data.loc[data['capacity'] == 'Itel it9211', 'capacity'] = convert_to_gb('16 MB')

Đối với các thông số kỹ thuật chung không thay đổi giữa các máy cùng dòng (operating_system, display_technology, screen_resolution, SIM, size, bluetooth) xử lý các dòng có tỉ lệ thiếu >0 và <100% bằng cách điền bằng giá trị đầu tiên của dòng có cùng tên. Nhận thấy việc điền thiếu các thông số này không quá cần thiết nên không thực hiện tra cứu để điền các dòng máy thiếu 100%.

In [ ]:
#operating_system
data['operating_system_missing'] = data['operating_system'].isna().astype('int')

rate_operating_system_name = data.groupby('name')['operating_system_missing'].mean()

# In ra các tên điện thoại có tỷ lệ thiếu 'operating_system' lớn hơn 0 và nhỏ hơn 1
print(rate_operating_system_name[(rate_operating_system_name > 0) & (rate_operating_system_name < 1)])

data = data.drop(columns=['operating_system_missing'])

In [ ]:
for name_value, group in data.groupby('name'):
    total = len(group)
    missing = group['operating_system'].isna().sum()
    ratio = missing / total

    if 0 < ratio < 1:
        # Lấy giá trị operating_system đầu tiên không bị thiếu trong nhóm
        os_value = group['operating_system'].dropna().iloc[0]
        # Gán lại cho các dòng bị thiếu trong nhóm đó
        data.loc[(data['name'] == name_value) & (data['operating_system'].isna()), 'operating_system'] = os_value

In [ ]:
#display_technology
data['display_technology_missing'] = data['display_technology'].isna().astype('int')
rate_display_technology_name = data.groupby('name')['display_technology_missing'].mean()
print(rate_display_technology_name[rate_display_technology_name > 0])
data = data.drop(columns=['display_technology_missing'])

display_technology không có dòng máy nào có tỉ lệ thiếu >0 và <100% 

In [ ]:
#screen_resolution
data['screen_resolution_missing'] = data['screen_resolution'].isna().astype('int')
rate_screen_resolution_name = data.groupby('name')['screen_resolution_missing'].mean()
print(rate_screen_resolution_name[rate_screen_resolution_name > 0])
data = data.drop(columns=['screen_resolution_missing'])

screen_resolution không có dòng máy nào có tỉ lệ thiếu >0 và <100% 

In [ ]:
#SIM
data['SIM_missing'] = data['SIM'].isna().astype('int')
rate_SIM_name = data.groupby('name')['SIM_missing'].mean()
print(rate_SIM_name[rate_SIM_name > 0])
data = data.drop(columns=['SIM_missing'])

SIM không có dòng máy nào có tỉ lệ thiếu >0 và <100% 

In [ ]:
#size
data['size_missing'] = data['size'].isna().astype('int')
rate_size_name = data.groupby('name')['size_missing'].mean()
print(rate_size_name[rate_size_name > 0])
data = data.drop(columns=['size_missing'])

size không có dòng máy nào có tỉ lệ thiếu >0 và <100% 

In [ ]:
#bluetooth
data['bluetooth_missing'] = data['bluetooth'].isna().astype('int')
rate_bluetooth_name = data.groupby('name')['bluetooth_missing'].mean()
print(rate_bluetooth_name[rate_bluetooth_name > 0])
data = data.drop(columns=['bluetooth_missing'])

bluetooth không có dòng máy nào có tỉ lệ thiếu >0 và <100% 

In [ ]:
data.isnull().sum()

## Xử lý giá trị ngoại lai

In [ ]:
# Kiểm tra khoảng giá toàn cục
print("Khoảng giá toàn cục")
for k, v in price_range.items():
    print(f"{k:>5}: {int(v):,}")

In [ ]:
# Đếm số giá khác nhau trong từng dòng máy
unique_prices_per_model = data.groupby('name')['price_new'].nunique().sort_values(ascending=False)
print("\nSố lượng giá khác nhau trong mỗi dòng máy:")
print(unique_prices_per_model)

In [ ]:
# Đếm số mẫu mỗi dòng máy
sample_count_per_model = data['name'].value_counts()
print("\nSố mẫu mỗi dòng máy:")
print(sample_count_per_model)

Giá và thông số khác biệt lớn giữa các dòng máy (từ vài trăm nghìn đến gần 50 triệu).
Đa số dòng máy có giá giao động trong khoảng nhất định.
Số lượng mẫu không đều giữa các dòng, xử lý toàn cục sẽ lệch kết quả.
-> xử lý theo từng dòng máy (name).